![image.png](attachment:image.png)

# Initialization

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor
import pandas as pd

matplotlib.use('TkAgg')

# 1. Show that this is an invertible map. Given any set $S \subset \mathbb{R}^2$ with positive measure, can you compute the limit of the measure of $H^{(n)}(S)$ when $n\rightarrow \infty$?

Let us demonstrate the invertibility of the map $H$. In fact, we will establish that $H$ is a diffeomorphism, which will prove useful for the second part of the exercise.

Given two manifolds, $M$ and $N$, a differential map $f: M \rightarrow N$ is referred to as a diffeomorphism if it is a bijective function, and its inverse $f^{-1}: N \rightarrow M$ is also differentiable.

It is evident that our function $H$ qualifies as a differential map. The only remaining task is to establish that $H$ is both a bijection and possesses a differentiable inverse. To accomplish this, we will employ the Inverse Function Theorem (IFT). To apply the theorem, we need to verify that the determinant of the Jacobian matrix is non-zero at some point. 

$$
\operatorname{det}\left(J_H(x, y)\right)=\operatorname{det}\left[\begin{array}{cc}
-2 a x & 1 \\
b & 0
\end{array}\right]=-b \neq 0 .
$$

Therefore, according to the IFT, the map $H$ has a locally continuously differentiable inverse at every point in $\mathbb{R}^2$. Notably, if we prove that $H$ is an injective map, it will establish that $H$ is a diffeomorphism, since injectivity implies the existence of a globally continuously differentiable inverse. To prove that $H$ is injective, let's assume that $H(x, y)=H\left(x', y'\right)$ and observe that $(x, y)=\left(x', y'\right)$. Hence, $H$ is injective, and consequently, $H$ is a diffeomorphism.

Let's now proceed with the second part of the exercise. For any set $S \subset \mathbb{R}^2$ with positive measure, it's worth noting that

$$
\lim _{n \rightarrow \infty} \mu\left(H^n(S)\right)=\lim _{n \rightarrow \infty} \int_{H^n(S)} 1 d \mu=\lim _{n \rightarrow \infty}\left|\operatorname{det}\left(J_{H^{(n)}}\right)\right| \int_S 1 d \mu
$$

where the last equality is due to the change of variable theorem for diffeomorphisms. Notice that

$$
\left|\operatorname{det}\left(J_{H^{(n)}}\right)\right|=\left|\operatorname{det}\left(\prod_{k=1}^{n-1} J_H\left(H^{n-k}\right) J_H\right)\right|=b^n .
$$

Hence, since $b=0.3$

$$
\lim _{n \rightarrow \infty} \mu\left(H^n(S)\right)=\lim _{n \rightarrow \infty} b^n \int_S 1 d \mu= \begin{cases}0 & \text { if } S \text { is a set of finite measure } \\ \infty & \text { otherwise. }\end{cases}
$$

# 2. (Optional) Make a plot of the dynamics of $H$. Note that there is an attractor set for many initial conditions. Make a plot of this attracting set. Feel free to zoom into the structure of this attracting set.

In [ ]:
def henon_map(x, y, a=1.4, b=0.3):
    ''' dissipative Hénon_map function to update the points
    '''
    x_new = 1 + y - a*(x**2)
    y_new = b*x
    x_new = np.clip(x_new, -1e10, 1e10)   # For overflow problems
    y_new = np.clip(y_new, -1e10, 1e10)   # For overflow problems
    
    return x_new, y_new

  
def on_click(event):
    ''' on_click function to receive as input the click event
    '''
    x, y = event.xdata, event.ydata   # Receive the coordinates of the point clicked
    plt.plot(x, y, 'k')   # Plot the initial point
    for i in range(1000):
        x, y = henon_map(x, y)   # Updated to the next points in the map
        plt.plot(x, y, 'ko', markersize=0.1)   # Plot the next point
    plt.draw()

In [ ]:
# Define the global parameter alpha ---------------------------------------------------------
a = 1.4
b = 0.3

# Open a figure -----------------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(8,8))

# Setup the cursor widget -------------------------------------------------------------------
cursor = Cursor(ax, useblit=True, color='red', linewidth=0.1)

# Figure settings ---------------------------------------------------------------------------
plt.axhline(y=0, color='black', lw=0.5)
plt.axvline(x=0, color='black', lw=0.5)
ax.set_xlim([-1.5, 1.5])
ax.set_ylim([-1.5, 1.5])
ax.set_aspect('equal')
ax.set_title(f'Dissipative Hénon map with a={a:.2f}, b={b:.2f}')
plt.connect('button_press_event', on_click)
plt.show()

One of the resulting images obtained is presented here for better consultation

![Figure_1-2.png](attachment:Figure_1-2.png)

# 3. This map has a fixed point $p_0$ near $(0.63, 0.19)$. Compute $p_0$ and its stability.

We are interested in finding a fixed point for the dissipative hénon map close to $p_0$. Meaning that we are interested in a point $(x,y)$ such that 

$$henon\_map(x,y)=(x,y)$$ 

Thus we can apply a Newton method to find such a point. We can do so to find the zero of the function

$$f(x,y)=henon\_map(x,y)-(x,y)=0$$

To do so, we need to compute the Jacobian of such function. By taking the partial derivates the Jacobian can be computed as

$$J(x,y)=\begin{bmatrix} \frac{\partial f_x}{\partial x} & \frac{\partial f_x}{\partial y} \\ \frac{\partial f_y}{\partial x} & \frac{\partial f_y}{\partial y} \end{bmatrix} = \begin{bmatrix} -2ax & 1 \\ b & 0 \end{bmatrix} - \mathcal{I}$$

In [ ]:
# Redefine the henon_map function to use as input the coordinates in a vector x -------------
def henon_map(x, a=1.4, b=0.3):
    ''' henon_map function to update the points
    '''
    x_new = np.copy(x)   # This is needed because sometimes python applies to the point the transformation when used in a function
    x_new[0] = 1 + x[1] - a*(x[0]**2)
    x_new[1] = b*x[0]
    x_new = np.clip(x_new, -1e10, 1e10)   # For overflow problems
    
    return x_new

def f(x, a=1.4, b=0.3):
    ''' Function of which we want to find the zero
    '''
    x_new = np.copy(x)   # This is needed because sometimes python applies to the point the transformation when used in a function
    
    return henon_map(x_new, a, b)-x_new

# Define the function to compute the Jacobian -----------------------------------------------
def Jacobian_f(x, a=1.4, b=0.3):
    ''' Jacobian of the function f
    '''
    x_new = np.copy(x)   # This is needed because sometimes python applies to the point the transformation when used in a function

    return np.array([[-2*a*x[0], 1], [b, 0]]) - np.eye(2)

We can now use a fixed point Newton method in order to compute the fixed point

In [ ]:
def Newton_method(x0, a=1.4, b=0.3, max_iter=1000, tol=1e-12):
    x_1 = x0
    for i in range(max_iter):
        x_2 = x_1 - np.dot(np.linalg.inv(Jacobian_f(x_1, a, b)),f(x_1, a, b))       # Here computing the inverse using this function is not computationally optimal,
        if np.linalg.norm(x_2-x_1) < tol:                                                    # and in this case it could be done by hand. But, since it is not exactly the object
            print("The method converged to the solution: (", x_2[0], ",", x_2[1], ")")   #  of this exercise, we can use it, as it is conceptually equivalent
            print("The method converged in", i, "iterations")
            return x_2   # Found a fixed point
        x_1 = x_2 
    
    print("The fixed point Newton method did not converge to a solution")
    return [np.nan, np.nan]

# Apply the fixed point Newton method using an ititial guess from the plotted map -----------
p0 = np.array([0.63, 0.19])   # Initial guess
x_fixed = Newton_method(p0)

The method converged to the solution: ( 0.6313544770895048 , 0.1894063431268514 )
The method converged in 2 iterations


In [ ]:
# Define the function to compute the Jacobian -----------------------------------------------
def Jacobian_henon(x, a=1.4, b=0.3):
    ''' Jacobian of the function f
    '''
    x_new = np.copy(x)   # This is needed because sometimes python applies to the point the transformation when used in a function

    return np.array([[-2*a*x[0], 1], [b, 0]])

print("--------------------------------------------------------------------------")
print("The eigenvalues of the Jacobian in the fixed point are:")
eigval = np.linalg.eigvals(Jacobian_henon(x_fixed))
print(eigval)
eigvals_norm = abs(np.linalg.eigvals(Jacobian_henon(x_fixed)))
print("with norm:", eigvals_norm)
print("--------------------------------------------------------------------------")
print("The determinant of the Jacobian in the fixed point is:")
determinant = np.linalg.det(Jacobian_henon(x_fixed))
print(determinant)
print("The trace of the Jacobian in the fixed point is:")
print(eigval[0]+eigval[1])
print("--------------------------------------------------------------------------")

--------------------------------------------------------------------------
The eigenvalues of the Jacobian in the fixed point are:
[-1.92373886  0.15594632]
with norm: [1.92373886 0.15594632]
--------------------------------------------------------------------------
The determinant of the Jacobian in the fixed point is:
-0.30000000000000004
The trace of the Jacobian in the fixed point is:
-1.7677925358506132
--------------------------------------------------------------------------


Since one eigenvalue, $\lambda_1 = -1.9237$, has $|\lambda_1|>1$, and the other eigenvalue, $\lambda_2 = 0.1559$, has $|\lambda_2|<1$, the fixed point is a saddle point, meaning it is an unstable fixed point.

# 4. Compute an approximation to the Lyapunov exponent of the attractor for different (at least two) initial conditions. Explain the computational method and discuss the results.

In order to approximate the Lyapunov exponent of the attractor we can perform the following steps:

* Choose an initial point in the phase space of the attractor. Let's call it $(x_0, y_0)$
* Compute the trajectory of this initial point using the map equations $$\begin{cases} x_{n+1} = 1+y_n-ax_n^2 \\ y_{n+1} = bx_n \end{cases}$$ Iterate these equations for a sufficient number of iterations to allow the system to reach the attractor. Let $N$ be the number of iterations
* At each iteration, compute the difference between the current trajectory and a nearby trajectory that starts from a slightly perturbed initial point $(x_0+\epsilon, y_0+\epsilon)$ is a small perturbation.
* Track the evolution of the difference over the iterations using the following equation $$\Delta_n = \sqrt( (x_n-x_{n,\epsilon})^2+(y_n-y_{n,\epsilon})^2 )$$ where $\Delta_n$ represents the distance between the two trajectories at iteration $n$, and $x_{n,\epsilon}$ and $y_{n,\epsilon}$ are the coordinates of the perturbed trajectory.
* Compute the average logarithm of the distance over the iterations $$\lambda = \frac{1}{N}\sum_{n=1}^N ln\left( \frac{\Delta_{n+1}}{\Delta_n} \right)$$ This average logarithm corresponds to the estimated Lyapunov exponent for the attractor of the dissipative Hénon map. A positive Lyapunov exponent indicates chaotic behavior, where nearby trajectories diverge exponentially, while a negative Lyapunov exponent suggests convergence or stability.

Note that obtaining an accurate estimation of the Lyapunov exponent often requires a large number of iterations and careful consideration of numerical precision. Additionally, it may be necessary to average the Lyapunov exponent over multiple initial conditions to obtain a more reliable result.

In [ ]:
n_iter = 10000
epsilon = 1e-3

x_1 = np.zeros(2)
trajectory_1 = [x_1]
for i in range(n_iter-1):
    x_new = henon_map(trajectory_1[-1])
    trajectory_1.append(x_new)

x_2 = x_1 + np.ones(2)*epsilon
trajectory_2 = [x_2]
for i in range(n_iter-1):
    x_new = henon_map(trajectory_2[-1])
    trajectory_2.append(x_new)

Delta_n = np.zeros(n_iter)
for i in range(n_iter):
    Delta_n[i] = np.linalg.norm(trajectory_1[i]-trajectory_2[i])

sum = 0
for i in range(n_iter-1):
    sum += np.log(Delta_n[i+1]/Delta_n[i])

lamb = sum/n_iter
print("Estimated Lyapunov exponent:", lamb)

Estimated Lyapunov exponent: 0.00047418608391354227


# 5. (Optional) From the previous computation, it is possible to derive the full set of Lyapunov exponents of this attractor?

In order to address this question, we can make an observation that leads us to an interesting result. Assuming that the sum of the first and second characteristic exponents, denoted as $\lambda_1$ and $\lambda_2$ respectively, is equal to the natural logarithm of the absolute value of parameter 'b', as stated in the theoretical framework, we can easily derive the second Lyapunov exponent (SLE) and subsequently determine the full set of Lyapunov exponents for this attractor.

By utilizing this assumption, we can proceed with the calculations to obtain the SLE. This finding is significant as it allows us to gain further insights into the dynamic behavior of the system.